In [ ]:
import math
import re
import random
import sys
import time
import zstandard as zstd
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from typing import Union, Dict, List
from collections import Counter
from sklearn.metrics import precision_recall_curve
from sklearn.metrics.pairwise import cosine_similarity
from matplotlib import font_manager

In [ ]:
# 对输入字符串进行分词处理，将字符串转化为小写并用非单词字符进行分割
def tokenize(s: str) -> list:
    return re.split(r'\W+', s.lower())

In [ ]:
# 规范化输入字符串，移除多余的空白和非单词字符
def normalize(s: str) -> str:
    s0 = re.sub(r'\b\W+ | \W+\b', ' ', s.lower())
    return re.sub(r'\b[^\w\s]+\b', '', s0)

In [ ]:
# 字符串规范化类
class StringNormalizer:
    def __init__(self, no_punct: bool = False, keep_punct: bool = False):
        self.patterns_to_replace = []
        # 根据参数决定是否去除标点符号
        if no_punct:
            self.patterns_to_replace += [('[¿?#!¡_]+|\.\.\.+', ' ')]
        else:
            if keep_punct is False:
                self.patterns_to_replace += [
                    ('\.\.\.+', ' ___ '),  # 替换省略号
                    ('[¿?]+', ' _?_ '),    # 替换问号
                    ('[!¡]+', ' _!_ '),    # 替换感叹号
                    ('#', " _#_ "),        # 替换井号
                ]
        
        # 定义其他替换模式
        self.patterns_to_replace += [
            ('[^\w\d?!_*&|#]+', ' '),  # 移除非单词字符
            ("(?<=\w)[’']+(?=\w)|(?<=\d)[,._\s]+(?=\d)", ''),  # 处理数字间的标点
            ('(?<!\d)\d\d?(?!\d)', ' ## '),  # 替换小于三位的数字
            ('(?<!\d)\d+(?!\d)', ' ### '),  # 替换多位数字
            ('\s+', ' ')  # 替换多个空格为一个空格
        ]

    # 规范化字符串
    def normalize_str(self, text: str):
        t = f' {text} '  # 在字符串开头和结尾添加空格以便后续处理
        for p in self.patterns_to_replace:
            t = re.sub(p[0], p[1], t)  # 应用所有替换模式
        return t[1:-1]  # 返回处理后的字符串，去除前后的空格

    # 从规范化字符串中提取单词
    def get_words(self, text: str):
        norm_text = self.normalize_str(text)
        return re.findall('\W*(\w+)', norm_text)  # 使用正则表达式提取单词

    # 获取单词计数
    def get_word_count(self, text: str):
        return len(self.get_words(text))  # 返回单词数量


In [ ]:
# 字符计数类
class CharCounter:
    def __init__(self,
                 samples: list,
                 name='',
                 IDF_thresh: float = 0.005,
                 max_samples: int = 50000):
        self.name = name
        self.IDF_thresh = IDF_thresh
        self.num_samples = min(max_samples, len(samples))  # 限制样本数
        self.populate_char_ctrs(samples[:max_samples], IDF_thresh)  # 填充字符计数器

        print(f"{len(self.good_chars)} good characters: ", self.good_chars)
        print(f"{len(self.leftover_chars)} leftover characters: ", self.leftover_chars)

        prob = self.compute_total_char_prob()
        if prob < 0.99:
            print("WARNING: In CharCounter, total probability is", prob, "(should be 1.0)")

    # 返回字符计数器的字符串表示
    def __str__(self):
        return (f"Character counter for *{self.name}* based on {self.num_samples} samples"
                f" and {self.num_chars} characters"
                f"\nTop characters by DF with threshold {100 * self.IDF_thresh:.3}% "
                f"(blank + word characters):\n {self.ctr_df_trimmed}"
                f"\nTop characters by occurrence:\n {self.ctr_occ_trimmed}"
                f"\nLeftover characters: {self.leftover_chars}")

    # 获取字符计数
    @staticmethod
    def get_char_counts(titles: list):
        ctr_df = Counter()  # 计数器，记录字符的出现频率
        ctr_occ = Counter()  # 计数器，记录字符的总出现次数
        OK_chars = re.compile('[^\w\d .,:;¿?¡!_*+/|#]+')  # 允许的字符正则表达式
        for t in titles:
            try:
                tn = re.sub(OK_chars, '', t)  # 移除不允许的字符
            except TypeError:
                print(t)
                print(type(t))
                return
            ctr_df += Counter(set(tn))  # 统计不重复字符
            ctr_occ += Counter(tn)  # 统计字符的出现次数
        return ctr_df, ctr_occ

    # 填充字符计数器
    def populate_char_ctrs(self, samples: list, IDF_thresh: float):
        n = len(samples)
        self.ctr_df, self.ctr_occ = self.get_char_counts(samples)  # 获取字符计数
        print("Found", len(self.ctr_df), "different characters (words, punctuation, blanks), not including emojis")
        self.ctr_df_trimmed = Counter()  # 修剪后的字符计数器
        self.ctr_occ_trimmed = Counter()  # 修剪后的字符出现次数计数器
        for chr, count in self.ctr_df.most_common(int(10 / IDF_thresh)):  # 根据阈值修剪字符
            self.ctr_df_trimmed[chr] = count
            self.ctr_occ_trimmed[chr] = self.ctr_occ[chr]
            self.min_occ = self.ctr_occ[chr]
            if count < n * IDF_thresh: break
        self.num_chars = sum(self.ctr_occ_trimmed.values())  # 计算总字符数
        self.good_chars = ''.join([c for c in self.ctr_df_trimmed])  # 好字符集合
        self.leftover_chars = ''.join([c for c in self.ctr_df - self.ctr_df_trimmed])  # 剩余字符集合
        self.bad_chars_regex = re.compile('[^' + self.good_chars + ']+')  # 不好的字符正则表达式

    # 移除不好的字符
    def remove_bad_chars(self, s: str):
        return re.sub('\s+', ' ', re.sub(self.bad_chars_regex, '', s))

    # 计算交叉熵
    def compute_cross_entropy(self, s: str):
        if len(s) == 0: return 0.0
        sum_lp = 0
        for c in s:
            p = self.get_char_prob(c)
            sum_lp += math.log(p)
        return -sum_lp / len(s)

    # 计算基于好字符的交叉熵
    def compute_cross_entropy1(self, s: str):
        s1 = self.remove_bad_chars(s)  # 移除不好的字符
        if len(s1) == 0: return 0.0
        sum_lp = 0
        for c in s1:
            p = self.get_char_prob(c)
            sum_lp += math.log(p)
        return -sum_lp / len(s1)

    # 获取字符的概率
    def get_char_prob(self, c: str):
        count = self.ctr_occ_trimmed[c]  # 获取字符出现次数
        if count == 0:
            count = self.min_occ / 2.0  # 若字符未出现，则使用最小出现次数的一半
        return count / self.num_chars  # 返回字符概率

    # 计算字符的总概率
    def compute_total_char_prob(self):
        sum = 0.0
        for c in self.ctr_occ_trimmed:
            sum += self.get_char_prob(c)
        return sum  # 返回字符总概率

In [ ]:
# 单词计数类
class WordCounter:
    def __init__(self,
                 samples: list,
                 name='',
                 IDF_thresh: float = 0.0001,
                 max_samples: int = 50000):
        self.name = name
        self.IDF_thresh = IDF_thresh
        self.num_samples = min(max_samples, len(samples))  # 限制样本数
        self.populate_word_ctrs(samples[:max_samples], IDF_thresh)  # 填充单词计数器

        prob = self.compute_total_word_prob()
        if prob < 0.99:
            print("WARNING: In WordCounter, total probability is", prob, "(should be 1.0)")

    # 返回单词计数器的字符串表示
    def __str__(self):
        return (f"Word counter for *{self.name}* based on {self.num_samples} samples"
                f" and {self.num_words} words"
                f"\nTop words by DF with threshold {100 * self.IDF_thresh:.3}% : {self.ctr_df_trimmed}"
                f"\nTop words by occurrence:\n {self.ctr_occ_trimmed}")

    # 获取单词计数
    @staticmethod
    def get_word_counts(titles: list):
        ctr_df = Counter()  # 计数器，记录单词的出现频率
        ctr_occ = Counter()  # 计数器，记录单词的总出现次数
        for t in titles:
            words = t.split()  # 按空格分割单词
            ctr_df += Counter(set(words))  # 统计不重复单词
            ctr_occ += Counter(words)  # 统计单词的出现次数
        return ctr_df, ctr_occ

    # 填充单词计数器
    def populate_word_ctrs(self, samples: list, IDF_thresh: float):
        n = len(samples)
        self.ctr_df, self.ctr_occ = self.get_word_counts(samples)  # 获取单词计数
        print("Found", len(self.ctr_df), "different words")
        self.ctr_df_trimmed = Counter()  # 修剪后的单词计数器
        self.ctr_occ_trimmed = Counter()  # 修剪后的单词出现次数计数器
        for w, count in self.ctr_df.most_common(int(10 / IDF_thresh)):  # 根据阈值修剪单词
            self.ctr_df_trimmed[w] = count
            self.ctr_occ_trimmed[w] = self.ctr_occ[w]
            self.min_occ = self.ctr_occ[w]
            if count < n * IDF_thresh: break
        self.num_words = sum(self.ctr_df_trimmed.values())  # 计算总单词数
        print(f"Trimmed down to {len(self.ctr_df_trimmed)} words")

    # 移除罕见单词
    def remove_rare_words(self, s: str):
        words = s.split()
        new_words = [w for w in words if self.ctr_df_trimmed[w] > 0]  # 保留出现次数大于0的单词
        return ' '.join(new_words)  # 返回新字符串

    # 计算单词的交叉熵
    def compute_cross_entropy(self, s: str):
        if len(s) == 0: return 0.0
        words = s.split()
        sum_lp = 0
        for c in words:
            p = self.get_word_prob(c)
            sum_lp += math.log(p)
        return -sum_lp / len(words)

    # 计算基于频繁单词的交叉熵
    def compute_cross_entropy1(self, s: str):
        if len(s) == 0: return 0.0
        words = s.split()
        new_words = [w for w in words if self.ctr_df_trimmed[w] > 0]  # 保留出现次数大于0的单词
        if len(new_words) == 0: return 0.0
        sum_lp = 0
        for c in new_words:
            p = self.get_word_prob(c)
            sum_lp += math.log(p)
        return -sum_lp / len(new_words)

    # 获取单词的概率
    def get_word_prob(self, word: str):
        count = self.ctr_occ_trimmed[word]  # 获取单词出现次数
        if count == 0:
            count = self.min_occ / 2  # 若单词未出现，则使用最小出现次数的一半
        return count / self.num_words  # 返回单词概率

    # 计算单词的总概率
    def compute_total_word_prob(self):
        sum = 0.0
        for word in self.ctr_occ_trimmed:
            sum += self.get_word_prob(word)
        return sum  # 返回单词总概率


In [ ]:
# 清理字符串，使用字符串规范化器和字符计数器
def cleanup_str(s: str, string_normalizer=None, char_ctr=None):
    return string_normalizer.normalize_str(char_ctr.remove_bad_chars(s))

# 清理语料库，遍历每个样本
def cleanup_corpus(corpus: list, string_normalizer=None, char_ctr=None):
    return [cleanup_str(s, string_normalizer, char_ctr) for s in corpus]  # 返回清理后的语料库

In [ ]:
def get_hashed_words(s: str):
    """
    将输入字符串中的每个单词转换为其哈希值的字节表示。
    该函数返回一个字节序列，每个单词用其哈希值的字节表示。
    
    :param s: 输入字符串
    :return: 表示单词哈希值的字节序列
    """
    bytes_list = []  # 用于存储每个单词的字节表示
    for w in s.split():
        if w == '': 
            continue  # 跳过空字符串
        h = hash(w)  # 获取单词的哈希值
        while h > 0:
            # 将哈希值转换为字节，范围为1到255
            bytes_list.append(1 + h % 255)  # 255而不是256，以便0可以作为单词分隔符
            h = h // 255  # 更新哈希值，向下取整
        bytes_list.append(0)  # 添加字节0作为结束符
    return bytes(bytes_list)  # 返回字节序列

In [ ]:
class ZstdLangModel:
    """
    Zstandard语言模型，用于文本压缩。
    假设输入已经过预处理（归一化等）。
    """

    ZSTD_LEVEL = 22  # 压缩级别
    dict_size_factor = 3 / 5  # 字典大小因子
    verbose = 0  # 是否输出详细信息
    max_num_dicts = 7  # 最大字典数量
    min_dict_size = 150  # 最小字典大小（字节）

    def __init__(self, name: str, corpus: list, primary_dict_size: int = 0, hash_words: bool = False):
        """
        初始化Zstd语言模型。

        :param name: 模型名称
        :param corpus: 训练语料库
        :param primary_dict_size: 主字典大小
        :param hash_words: 是否对单词进行哈希处理
        """
        if self.verbose > 0:
            print(f'For the "{name}" corpus of size {len(corpus)}: ')
            not_str = '' if hash_words else ' not'
            print(f'  * words are{not_str} hashed ')  # 输出是否对单词进行哈希处理
        self.name = name  # 保存模型名称
        self.primary_dict_size = primary_dict_size  # 设置主字典大小
        self.hash_words = hash_words  # 设置是否哈希单词

        self.set_params(corpus)  # 设置参数
        self.train_dicts(corpus)  # 训练字典

    def set_params(self, corpus: list):
        """
        设置模型参数，包括主字典大小和字典数量。

        :param corpus: 训练语料库
        """
        # 如果需要哈希单词，则生成字节对象列表
        if self.hash_words:
            self.bytes_obj_list = [get_hashed_words(s) + b'|' for s in corpus]
            corpus_b = b''.join(self.bytes_obj_list)  # 连接所有字节
        else:
            corpus_b = bytes('\n'.join(corpus), 'utf8')  # 将文本语料库转换为字节

        orig_size = len(corpus_b)  # 记录原始大小
        self.c = zstd.ZstdCompressor(level=self.ZSTD_LEVEL)  # 创建压缩器
        compr_size = len(self.c.compress(corpus_b))  # 获取压缩后的大小
        del corpus_b  # 删除临时字节对象以释放内存

        # 如果未指定主字典大小，则根据压缩后的大小计算
        if self.primary_dict_size == 0:
            self.primary_dict_size = int(self.dict_size_factor * compr_size)

        # 计算字典数量
        self.num_dicts = max(min(
            self.max_num_dicts,
            int(math.log2(self.primary_dict_size / self.min_dict_size))), 1)

        if self.verbose > 0:
            print(
                f'  * compression rate = {compr_size / orig_size:.4}, compr. size {compr_size}'
            )  # 输出压缩率

    def train_dicts(self, corpus: list):
        """
        训练字典。

        :param corpus: 训练语料库
        """
        samples = self.bytes_obj_list if self.hash_words else [
            bytes(s, 'utf8') for s in corpus  # 将语料库转换为字节格式
        ]

        def get_dict(j: int, primary_dict_size: int, samples: list):
            """
            生成指定大小的训练字典。

            :param j: 字典索引
            :param primary_dict_size: 主字典大小
            :param samples: 样本数据
            :return: 训练生成的字典
            """
            sz = max(int(primary_dict_size / 2**j), 256)  # 确定字典大小
            return zstd.train_dictionary(sz, samples)  # 训练并返回字典

        if self.verbose:
            print(
                f'  * building {self.num_dicts} dictionaries of size {self.primary_dict_size} and smaller'
            )  # 输出正在构建的字典数量和大小

        # 生成字典列表
        self.dicts = [
            get_dict(j, self.primary_dict_size, samples)
            for j in range(self.num_dicts)
        ]

        # 创建压缩器上下文，忽略各种头部信息
        self.cctx = [
            zstd.ZstdCompressor(
                dict_data=d,
                write_content_size=False,
                write_dict_id=False,
                write_checksum=False,
                level=self.ZSTD_LEVEL) for d in self.dicts
        ]

        self.header_size = len(self.cctx[0].compress(b''))  # 计算压缩头部大小
        if self.verbose > 0:
            print('  * compression headers of size', self.header_size,
                  'will be neglected')  # 输出将忽略的头部大小

        if self.hash_words: 
            del self.bytes_obj_list  # 删除字节对象列表以释放内存

        return

    def get_compr_size(self, t: str):
        """
        获取给定文本的压缩大小。

        :param t: 输入文本
        :return: 压缩大小和原始大小
        """
        t_b = bytes(t, 'utf8')  # 将文本转换为字节
        l_orig = len(t_b)  # 原始大小
        if l_orig < 5: 
            return l_orig, l_orig  # 若文本长度小于5，则直接返回原始大小

        if self.hash_words:
            t_b = get_hashed_words(t)  # 对文本进行哈希处理

        # 计算压缩后的大小
        l_compr = sum([len(c.compress(t_b))
                       for c in self.cctx]) / len(self.cctx) - self.header_size
        l_orig = len(self.c.compress(t_b)) - self.header_size  # 计算原始压缩大小
        return l_compr, l_orig  # 返回压缩大小和原始大小

In [ ]:
class Zstd2Classifier:
    """
    Zstandard分类器，使用ZstdLangModel进行分类。
    假设输入已经过预处理（归一化等）。
    """

    verbose = 0  # 是否输出详细信息

    def __init__(self, name: str, pos_samples: list, neg_samples: list, hash_words: bool = False):
        """
        初始化分类器。

        :param name: 分类器名称
        :param pos_samples: 正样本列表
        :param neg_samples: 负样本列表
        :param hash_words: 是否对单词进行哈希处理
        """
        if self.verbose > 0:
            print(
                f'Building a "{name}" classifier for {len(pos_samples)} positive and '
                f'{len(neg_samples)} negative samples')  # 输出构建分类器的信息

        # 创建正负样本的Zstd语言模型
        self.pos_model = ZstdLangModel('positive', pos_samples, 0, hash_words)
        self.neg_model = ZstdLangModel('negative', neg_samples,
                                       self.pos_model.primary_dict_size, hash_words)

    def score(self, s: str, num_reps: int = 2):
        """
        对给定字符串进行评分。

        :param s: 输入字符串
        :param num_reps: 字符串重复次数
        :return: 分类得分、负样本压缩率、正样本压缩率
        """
        s_x = s * num_reps  # 重复输入字符串
        # 获取正负样本的压缩大小和原始大小
        compr_sz_pos, orig_sz_pos = self.pos_model.get_compr_size(s_x)
        compr_sz_neg, orig_sz_neg = self.neg_model.get_compr_size(s_x)
        assert orig_sz_pos == orig_sz_neg, 'Two orig size estimates differ'  # 确保原始大小一致
        if orig_sz_pos == 0:
            return 0.0, 1.0, 1.0  # 若原始大小为0，返回默认值

        # 计算正负样本的压缩率
        pos_rate = compr_sz_pos / orig_sz_pos
        neg_rate = compr_sz_neg / orig_sz_neg

        score = neg_rate - pos_rate  # 计算分类得分
        return score, neg_rate, pos_rate  # 返回得分和压缩率